In [ ]:
import pandas as pd
import numpy as np
from statsmodels.graphics.tsaplots import plot_acf

import seaborn as sns
from scipy.stats import norm
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
def calc_supply_demand(df):
    #print(df)
    df = df.reset_index(level=0, drop=True)
    df = df.reindex(np.arange(df.index.min(), df.index.max() + 1))
    df = df.sort_index().fillna(0)
    df["supply"] = df["ask"].sort_index(ascending=True).cumsum()
    df["demand"] = df["bid"].sort_index(ascending=False).cumsum()
    return df

def plot_market_depth(df_book, price_range=None):
    #%matplotlib notebook
    
    df = df_book.pivot_table(
        columns="side",
        index=["day", "price"],
        values="quantity",
        aggfunc="sum"
    ).groupby("day").apply(calc_supply_demand)
    
    df_plot = df[["supply", "demand"]].stack().rename("quantity").reset_index().sort_values(["day", "price"])
    df_plot = df_plot.groupby(["day", "price"])["quantity"].sum().reset_index()

    df_plt = pd.DataFrame()
    for day in df_plot["day"].unique():
        df_day = df_plot[df_plot["day"] == day]
        df_day = (
            df_day
            .sort_values("price")
            .set_index("price")
            .reindex(np.arange(df_plot["price"].min(), df_plot["price"].max() + 1))
        ).bfill().ffill()

        if price_range is not None:
            df_day = df_day[(df_day.index >= price_range[0]) & (df_day.index <= price_range[1])]

        df_plt = df_plt.append(df_day)

    df_plt = df_plt.reset_index()
    x = df_plt["day"]
    y = df_plt["price"]
    z = df_plt["quantity"]



    fig = plt.figure()
    ax = fig.gca(projection='3d')
    surf = ax.plot_trisurf(x, y, z, cmap=cm.coolwarm) # , linewidth=0.1
    #plt.clim(0,4)
    ax.set_zlim(0,df_plt["quantity"].max())
    #ax.set_ylim(50,100)
    return fig

## Basic

In [ ]:
%matplotlib inline
df_trd = pd.read_csv("results/basic_lower/trades.csv")
df_trd = df_trd.sort_values("timestamp")

fig, axs = plt.subplots(3, 1, figsize=[7,10], sharex=True)

df_trd.set_index("trading_day")["price"].plot(ax=axs[0])
#axs[0].axhline(1000)
df_trd.groupby("trading_day")["quantity"].sum().plot(ax=axs[1]) # , kind="bar"

(df_trd.groupby("trading_day")["quantity"].sum().reindex(np.arange(1, 1001)) * df_trd.groupby("trading_day")["price"].mean().reindex(np.arange(1, 1001))).plot(ax=axs[2])

ax.set_xlabel("Trading session")
axs[0].set_ylabel("Price")
axs[1].set_ylabel("Traded quantity (sum per session)")
axs[2].set_ylabel("Traded value (sum per session)")
fig.suptitle('Trades throughout the simulation', fontsize=14)
fig.savefig(f"../thesis/plots/basic_trades_lower.png")

In [ ]:
axs[0, 1]

In [ ]:
%matplotlib inline

fig, axs = plt.subplots(3, 2, figsize=[10,10], sharex='col', sharey='row')

df_trd = pd.read_csv("results/basic_lower/trades.csv")
df_plot_lower = pd.DataFrame(index=df_trd.trading_day.unique())
df_plot_lower["price"] = df_trd.groupby("trading_day")["price"].mean()
df_plot_lower["quantity"] = df_trd.groupby("trading_day")["quantity"].sum()
df_plot_lower["value"] = df_plot_lower["price"] * df_plot_lower["quantity"]

df_trd = pd.read_csv("results/basic_higher/trades.csv")
df_plot_higher = pd.DataFrame(index=df_trd.trading_day.unique())
df_plot_higher["price"] = df_trd.groupby("trading_day")["price"].mean()
df_plot_higher["quantity"] = df_trd.groupby("trading_day")["quantity"].sum()
df_plot_higher["value"] = df_plot_higher["price"] * df_plot_higher["quantity"]

for i, df_trd in enumerate((
    pd.read_csv("results/basic_lower/trades.csv"), 
    pd.read_csv("results/basic_higher/trades.csv")
)):
    df = pd.DataFrame(index=df_trd.trading_day.unique())
    df["price"] = df_trd.groupby("trading_day")["price"].mean()
    df["quantity"] = df_trd.groupby("trading_day")["quantity"].sum()
    df["value"] = df_plot_higher["price"] * df_plot_higher["quantity"]
    
    df["price"].plot(ax=axs[0, i])
    #axs[0].axhline(1000)
    df["quantity"].plot(ax=axs[1, i]) # , kind="bar"

    df["value"].plot(ax=axs[2, i])

    ax.set_xlabel("Trading session")
    axs[0, i].set_ylabel("Price")
    axs[1, i].set_ylabel("Traded quantity (sum per session)")
    axs[2, i].set_ylabel("Traded value (sum per session)")

    axs[0, i].set_title({0: "Market price starting from 500", 1: "Market price starting from 1 500"}[i])

axs[2, 0].set_xlabel("Trading session")
axs[2, 1].set_xlabel("Trading session")
fig.savefig(f"../thesis/plots/basic_trades.png")

In [ ]:
%matplotlib notebook

In [ ]:
df_books = pd.read_csv("results/basic_lower/books.csv").set_index("day")
df_books = df_books.loc[0:100]
#%matplotlib notebook
fig = plot_market_depth(df_books)

#plt.title("Market depth")
plt.xlabel("Trading session")
plt.ylabel("Price")
plt.gca().set_zlabel('Quantity')

plt.gca().view_init(60, -30)
plt.margins(0, 0, 0)
#df_trd.groupby("trading_day")["price"].mean().loc[0:100].plot()

In [ ]:
fig.savefig(f"../thesis/plots/basic_market_depth_converge_lower.png")

In [ ]:
df_books = pd.read_csv("results/basic_higher/books.csv").set_index("day")
df_books = df_books.loc[0:100]

fig = plot_market_depth(df_books)

#plt.title("Market depth")
plt.xlabel("Trading session")
plt.ylabel("Price")
plt.gca().set_zlabel('Quantity')

plt.gca().view_init(60, -30)
plt.margins(0, 0, 0)
#df_trd.groupby("trading_day")["price"].mean().loc[0:100].plot();

In [ ]:
fig.savefig(f"../thesis/plots/basic_market_depth_converge_higher.png")

In [ ]:
# Autocorrelation of results
df_trd = pd.read_csv("results/basic_lower/trades.csv")
df_trd = df_trd[df_trd["trading_day"] >= 100].sort_values("timestamp")
#df_prices = df_trd.sort_values("timestamp").groupby("trading_day")["price"].last().iloc[50:]

fig = plot_acf(df_trd["price"].pct_change(), lags=7, title=f"Autocorrelation of Rate of Returns")
fig.savefig(f"../thesis/plots/basic_autocorrelation.png")

In [ ]:
# Vola clusters
df_trd = pd.read_csv("results/basic_lower/trades.csv")
df_trd = df_trd[df_trd["trading_day"] >= 100].sort_values("timestamp")
fig = plot_acf(df_trd["price"].pct_change().groupby(df_trd["price"].index // 10).std(), lags=10, title=f"Autocorrelation of volatility (10 trades)")
fig.savefig(f"../thesis/plots/basic_volaclusters.png")

In [ ]:
# Fat tails
df_trd = pd.read_csv("results/basic_lower/trades.csv")
df_trd = df_trd[df_trd["trading_day"] >= 100].sort_values("timestamp")

fig = plt.figure()
#data = norm.rvs(5,0.4,size=1000) # you can use a pandas series or a list if you want

ax =sns.distplot(df_trd["price"].pct_change().dropna(), fit=norm, kde=False)
ax.set_title(f"Distribution of returns & fitted normal distribution")
plt.legend(["Fitted normal distribution", "Simulation"])
ax.set_xlabel("Rate of returns")
fig.savefig("../thesis/plots/basic_fat_tails.png")